In [1]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential

from keras.layers import Dense, Dropout
# from keras.layers.normalization import BatchNormalization
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [4]:
df_train = pd.read_csv('./data/asilomar_nov182017_all.csv')
# this is actually [u1, v1, u2, v2]
X = df_train[['u','v','w','h']]
y = df_train['distance']

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.30)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

In [6]:
NUM_FEATURES = len(X.columns)

In [7]:
def dist_regression_model():
    model = Sequential()
    model.add(Dense(NUM_FEATURES*4, input_dim=NUM_FEATURES, activation='relu'))
#     model.add(BatchNormalization())
    model.add(Dense(NUM_FEATURES*4, activation='relu'))
#     model.add(BatchNormalization())
    model.add(Dense(NUM_FEATURES*4, activation='relu'))
#     model.add(BatchNormalization())
    model.add(Dense(NUM_FEATURES*4, activation='relu'))
#     model.add(BatchNormalization())
    model.add(Dense(NUM_FEATURES*4, activation='relu'))
#     model.add(BatchNormalization())
    model.add(Dense(NUM_FEATURES*4, activation='relu'))
    model.add(Dense(NUM_FEATURES*4, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [19]:
model = dist_regression_model()
model.fit(x=X_train, 
          y=y_train, 
          batch_size=32, 
          epochs=50, 
          validation_data = (X_val, y_val))

Train on 11088 samples, validate on 4753 samples
Epoch 1/50
11088/11088 [==============================] - 1s 67us/step - loss: 993.1515 - val_loss: 169.9807
Epoch 2/50
11088/11088 [==============================] - 0s 45us/step - loss: 159.9531 - val_loss: 153.9270
Epoch 3/50
11088/11088 [==============================] - 1s 55us/step - loss: 146.2040 - val_loss: 141.5162
Epoch 4/50
11088/11088 [==============================] - 1s 54us/step - loss: 132.5369 - val_loss: 127.7198
Epoch 5/50
11088/11088 [==============================] - 0s 45us/step - loss: 122.9758 - val_loss: 121.6174
Epoch 6/50
11088/11088 [==============================] - 1s 46us/step - loss: 116.8196 - val_loss: 116.7652
Epoch 7/50
11088/11088 [==============================] - 0s 44us/step - loss: 111.7303 - val_loss: 114.5468
Epoch 8/50
11088/11088 [==============================] - 0s 43us/step - loss: 106.7702 - val_loss: 114.0310
Epoch 9/50
11088/11088 [==============================] - 0s 44us/step - loss: 

In [20]:
def evaluate_model(model, X_val, y_val):
    # training error
    pred = model.predict(X_val)

    error = y_val[:,np.newaxis]-pred
    print 'MSE: {}'.format(np.mean(np.square(error)))

    # mean absolute percentage
    mean_abs_percentage = np.mean(np.abs(error)/y_val[:,np.newaxis])
    print 'Mean Abs Percentage:{}'.format(mean_abs_percentage)

evaluate_model(model, X_val, y_val)

MSE: 95.4196220838
Mean Abs Percentage:0.115043921209


In [21]:
# test set
X_val_np = X_val
y_val_np = y_val
pred = model.predict(X_val_np)

# mean square error
error = y_val_np[:,np.newaxis]-pred
print np.mean(np.square(error))

# mean absolute percentage
mean_abs_percentage = np.mean(np.abs(error)/y_val_np[:,np.newaxis])
print mean_abs_percentage

95.41962208378774
0.11504392120861687


In [22]:
print error[:10]

[[ 1.31268547e+00]
 [ 6.89054445e-03]
 [-2.67833434e+00]
 [ 7.25586408e+00]
 [-1.17855883e-01]
 [ 2.95889232e+00]
 [ 2.49069443e-01]
 [ 1.66301307e+00]
 [-3.16023579e-01]
 [ 2.53417482e+00]]


In [13]:
bbox = np.array([[147.,  295.,147+331.,295.+173.]],np.float32)
bbox = scaler.transform(bbox)
print bbox.shape
print model.predict(bbox)

(1, 4)
[[13.313145]]


In [14]:
scaler.inverse_transform(X_train[100,:])

array([107., 255., 402., 475.])

In [15]:
bbox = np.array([[240,300,46,39]],np.float32)
scaler.transform(bbox)

array([[-0.1473077,  4.6086993, -4.7980194, -5.175077 ]], dtype=float32)